In [1]:
import numpy as np
import os
import fnmatch
from xml.etree.ElementTree import ElementTree
import xml.etree.ElementTree as ET
from struct import unpack
import binascii
import shutil
from PIL import Image
import cv2

In [2]:
def hex_to_double(s):
    return unpack(">d", binascii.unhexlify(s))[0]

def getline(list1,list2, N):
    res = []   
    
    l1 = [(x, 0) for x in list1]
    l2 = [(x, 1) for x in list2]
    for l in l2:
        l1.append(l)
    line = l1
    line.sort()
    
    cnt=0
    val1 = 0
    for l in line:
        if cnt==0:
            res.append((val1, l[0]))
        if l[1]==0:
            cnt = cnt+1
        else:
            cnt = cnt-1
        if cnt==0:
            val1 = l[0]
    res.append((val1, N))
    
    return res

In [10]:
dir_path = r'D:\tablebank\20190424190300041510\marmot_dataset_v1.0\data\Chinese\Positive'
xml_dir = os.path.join(dir_path, 'Labeled')
img_dir = os.path.join(dir_path, 'Raw')


In [11]:
images = {}
file_list =  os.listdir(img_dir)
for file_name in fnmatch.filter(file_list, "*.bmp"):
    img_path = os.path.join(img_dir, file_name)   
    img = cv2.imread(img_path)
    height,width = img.shape[:2]
    name = os.path.splitext(os.path.split(img_path)[1])[0]
    images[name] = {"width": width, "height": height}

    xml_path = os.path.join(xml_dir, name+'.xml')
    marmot_tree = ET.parse(xml_path)
    marmot_root = marmot_tree.getroot()

    tables = marmot_root.findall("*/Composites/*[@Label='TableBody']")
    cells = marmot_root.findall("*/Composites/*[@Label='Textline']")
    
        
    cnt = 0
    for table in tables:
        hexs = table.get("BBox").split(" ")
        bbox_array = [int(hex_to_double(x)*1.33) for x in hexs]
        x0 = min(bbox_array[0], bbox_array[2])
        x1 = max(bbox_array[0], bbox_array[2])
        y0 = min(height - bbox_array[3],height - bbox_array[1])
        y1 = max(height - bbox_array[3],height - bbox_array[1])
        bbox_array[0] = x0
        bbox_array[2] = x1
        bbox_array[1] = y0
        bbox_array[3] = y1
        img_table = img[bbox_array[1]:bbox_array[3], bbox_array[0]:bbox_array[2]]

        save_path = os.path.join(dir_path, 'images/'+name+'_'+str(cnt)+'.jpg')
        cv2.imwrite(save_path, img_table)

        table_cell = []

        cell_x0 = []
        cell_x1 = []
        cell_y0 = []
        cell_y1 = []


        LID = table.get("LID")
        for cell in cells:
            lid = cell.get("PLID")
            if lid==LID:
                chexs = cell.get("BBox").split(" ")
                cbox_array = [int(hex_to_double(x)*1.33) for x in chexs]
                x0 = min(cbox_array[0], cbox_array[2])
                x1 = max(cbox_array[0], cbox_array[2])
                y0 = min(height - cbox_array[3],height - cbox_array[1])
                y1 = max(height - cbox_array[3],height - cbox_array[1])
                y0 = y0 - bbox_array[1]
                y1 = y1 - bbox_array[1]
                x0 = x0 - bbox_array[0]
                x1 = x1 - bbox_array[0]
                cbox_array = [x0,y0,x1,y1]
                table_cell.append(np.array(cbox_array))
                cell_x0.append(x0)
                cell_x1.append(x1)
                cell_y0.append(y0)
                cell_y1.append(y1)

        mask_h = np.zeros((img_table.shape[0], img_table.shape[1]))
        mask_v = np.zeros((img_table.shape[0], img_table.shape[1]))

        vlines = getline(cell_x0, cell_x1, img_table.shape[1])
        hlines = getline(cell_y0, cell_y1, img_table.shape[0])


        for l in hlines:
            cv2.rectangle(mask_h, (0,l[0]), (img_table.shape[1], l[1]), (1,0,0), -1) 
        save_path = os.path.join(dir_path, 'images/'+name+'_'+str(cnt)+'_maskh.png')
        cv2.imwrite(save_path, mask_h)
        for l in vlines:
            cv2.rectangle(mask_v, (l[0], 0), (l[1], img_table.shape[0]), (1,0,0), -1)
        save_path = os.path.join(dir_path, 'images/'+name+'_'+str(cnt)+'_maskv.png')
        cv2.imwrite(save_path, mask_v)            
        
        cnt = cnt+1

In [140]:
cv2.imshow('tmp',mask_h)
cv2.waitKey(10000)

255